In [25]:
# pre-process the label data
import pandas as pd
df=pd.read_csv("/Users/az/OneDrive - University of Houston Downtown/CS 4395 Senior Project/RNATracker-master/Data/cefra-seq/Supplemental_File_3.tsv", delimiter='\t')

# rename the column name
df.rename(columns={'ensembl_gene_id':'gene_id'}, inplace=True)

# slice needed columns
df=df.loc[:,['gene_id', 'cyto_A', 'cyto_B', 'insol_A', 'insol_B', 'membr_A', 'membr_B', 'nucl_A', 'nucl_B']]

# mutate a new avg column 
df['avg_cyto']=(df.cyto_A+df.cyto_B)/2
df['avg_insol']=(df.insol_A+df.insol_B)/2
df['avg_membr']=(df.membr_A+df.membr_B)/2
df['avg_nucl']=(df.nucl_A+df.nucl_B)/2

# mutate a new column with sum values
df['sum']=df.avg_cyto+df.avg_insol+df.avg_membr+df.avg_nucl

# mutate a new percent localization value column for each compartment
df['cyto_loc']=df.avg_cyto/df['sum']
df['insol_loc']=df.avg_insol/df['sum']
df['membr_loc']=df.avg_membr/df['sum']
df['nucl_loc']=df.avg_nucl/df['sum']

# select only necessary columns into df
df=df.loc[:,['gene_id', 'sum','cyto_loc', 'insol_loc', 'membr_loc', 'nucl_loc']]
print("tsv file size is: ", df.shape)

# compute the number or rows with zero localization values in all four compartments 
null_df=df[df['sum']==0]
print("{} rows have ZERO sum localization values: ".format(null_df.shape[0]))
print("{} rows have NON-zero local sum values ".format(df.shape[0]-null_df.shape[0]))

# filter out rows with non-zero local sum values
df=df[df['sum']!=0]

# drop 'sum' column
df=df.loc[:, df.columns != 'sum']
print("usable data from tsv file has shape ", df.shape) 
df.head()

tsv file size is:  (63677, 6)
35744 rows have ZERO sum localization values: 
27933 rows have NON-zero local sum values 
usable data from tsv file has shape  (27933, 5)


,gene_id,cyto_loc,insol_loc,membr_loc,nucl_loc
0,ENSG00000000003,0.404002,0.163435,0.239304,0.193258
2,ENSG00000000419,0.187355,0.450069,0.130292,0.232284
3,ENSG00000000457,0.128840,0.332606,0.166058,0.372495
4,ENSG00000000460,0.154924,0.300883,0.201207,0.342986
5,ENSG00000000938,0.000000,0.000000,0.000000,1.000000


In [26]:
# prepare the sequence data from the fasta file from cefra-seq dataset with BioPython library
import numpy as np
from Bio import SeqIO

fasta=[]
for record in SeqIO.parse('/Users/az/OneDrive - University of Houston Downtown/CS 4395 Senior Project/RNATracker-master/Data/cefra-seq/cefra_seq_cDNA_screened.fa', "fasta"):
    fasta.append({'gene_id': record.id, 'seq':record.seq})    

# convert a list of dictionaries into a dataframe
df_fasta=pd.DataFrame(fasta)

s=pd.Series(df_fasta['seq'])
# get the length of the longest sequence - 34526
print("the longest sequence has {} characters".format(max(s.str.len())))
# get the length of the shortest sequence - 207
print("the shortest sequence has {} characters".format(min(s.str.len())))

# create a list with length values of sequences
len_list=list(s.str.len())
# find the 75th percentile 
a = np.array(len_list)
p = np.percentile(a, 75) # return 75th percentile - 5060
print("\n75th percentile is ", p) 
p2 = np.percentile(a, 50) # return 50th percentile - 3303
print("50th percentile is ", p2) 
print("mean - average length: ", np.mean(a))
p3 = np.percentile(a, 90) # return 90th percentile - 7337.6
print("90th percentile is ", p3) 
p4 = np.percentile(a, 98) # return 98th percentile - 11552.79
print("98th percentile is ", p4) 
print("\ndf_fasta is: \n", df_fasta)

the longest sequence has 34526 characters
the shortest sequence has 207 characters

75th percentile is  5060.0
50th percentile is  3303.0
mean - average length:  3967.9226237580233
90th percentile is  7337.600000000002
98th percentile is  11552.799999999997

df_fasta is: 
                gene_id                                                seq
0      ENSG00000000003  (G, G, C, A, C, C, A, G, G, G, G, C, C, A, G, ...
1      ENSG00000000419  (A, T, T, C, C, G, C, T, T, C, C, G, G, C, A, ...
2      ENSG00000000457  (T, T, T, C, C, G, G, A, C, C, C, G, T, C, T, ...
3      ENSG00000000460  (G, G, C, T, T, T, G, G, C, C, C, T, G, G, A, ...
4      ENSG00000001036  (C, A, G, C, G, C, T, C, C, C, G, A, G, G, C, ...
...                ...                                                ...
11368  ENSG00000271303  (C, G, G, C, A, C, C, T, G, G, C, G, A, G, C, ...
11369  ENSG00000272047  (A, C, T, C, T, G, C, C, G, G, C, A, A, C, G, ...
11370  ENSG00000272325  (G, C, C, T, T, A, C, G, C, C, G, C,

In [27]:
# now merge 2 pandas dataframes: sequence dataframe and label dataframe 
df_merged = pd.merge(df, df_fasta, on='gene_id', how='inner')
print("merged dataframe tsv with fasta has shape: ", df_merged.shape)
# move the order of columns
data=df_merged[['gene_id', 'seq', 'cyto_loc', 'insol_loc', 'membr_loc', 'nucl_loc']]

# save a full_seq data into .csv file 
# df_merged.to_csv('full_seq_and_loc.csv', index=False)

#check how many unique genes we have in a merged dataframe
mrna=list(data['gene_id'])
print("the number of unique gene id-s: ", len(set(mrna)))

merged dataframe tsv with fasta has shape:  (11373, 6)
the number of unique gene id-s:  11373


In [28]:
# sort gene_id-s by the int number in gene_id
# create a new column with extracted integer numbers out of gene_id
data['sort'] = data['gene_id'].str.extract('(\d+)', expand=False).astype(int)
data.sort_values('sort',inplace=True, ascending=True)
# drop the "sort" column
data = data.drop('sort', axis=1)
data.head()

/Users/az/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/az/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,gene_id,seq,cyto_loc,insol_loc,membr_loc,nucl_loc
0,ENSG00000000003,"(G, G, C, A, C, C, A, G, G, G, G, C, C, A, G, ...",0.404002,0.163435,0.239304,0.193258
1,ENSG00000000419,"(A, T, T, C, C, G, C, T, T, C, C, G, G, C, A, ...",0.187355,0.450069,0.130292,0.232284
2,ENSG00000000457,"(T, T, T, C, C, G, G, A, C, C, C, G, T, C, T, ...",0.128840,0.332606,0.166058,0.372495
3,ENSG00000000460,"(G, G, C, T, T, T, G, G, C, C, C, T, G, G, A, ...",0.154924,0.300883,0.201207,0.342986
4,ENSG00000001036,"(C, A, G, C, G, C, T, C, C, C, G, A, G, G, C, ...",0.333751,0.156757,0.200905,0.308587


In [29]:
#prepare 8kb dataset: truncating='pre', padding='pre'

maxlen=4000
# truncating='pre' sequences longer than maxlen, keep only last maxlen characters, truncate at the beginning
for i in range(len(data.seq)):
    if len(data.seq[i])>maxlen:
        data.seq[i]=data.seq[i][-maxlen:]

# check the length of the previously longest sequence        
print(len(data.seq[8963]))    

/Users/az/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


4000


In [30]:
# One-Hot encoding and padding='pre'
from numpy import array
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# convert pandas Series into list 
seq=data["seq"].tolist()
mylist=[]
for item in seq:
    mylist.append(list(item))  
    
newlist=[]
label_encoder = LabelEncoder()
label_encoder.fit(np.array(['A','C','G','T']))

for seq in mylist:
    seq_array=np.array(seq)
    integer_encoded=label_encoder.transform(seq_array)
    onehot_encoded = to_categorical(integer_encoded)
    # np.pad with zeros sequences with length less than maxlen characters - padding='pre' or left padding
    onehot_encoded=np.pad(onehot_encoded, ((maxlen-onehot_encoded.shape[0],0),(0,0)), 'constant')
    newlist.append(onehot_encoded)

my_array=np.array(newlist) 
print("last 10 characters os the first sequence: ", data.seq[0][-10:])
# verify one-hot encoding of the first and last 10 characters of the first sequence
print("one-hot encoded version of first 10 characters\n",my_array[0][0:10])
print("one-hot encoded version of last 10 characters\n",my_array[0][-10:])
print("training data array shape: ", my_array.shape)

last 10 characters os the first sequence:  AATACAAGGT
one-hot encoded version of first 10 characters
 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
one-hot encoded version of last 10 characters
 [[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
training data array shape:  (11373, 4000, 4)


In [31]:
# write a .h5 file with one-hot encoded sequence data
import h5py
h5f = h5py.File('seq_data_4000_PRE.h5', 'w')
h5f.create_dataset('seq_data_4000_PRE', data=my_array)
h5f.close()

In [32]:
# read in .h5 file to check 
with h5py.File('seq_data_4000_PRE.h5', 'r') as hf:
    data = hf['seq_data_4000_PRE'][:]
    h5f.close()
data[0][-10:]

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)

In [16]:
#convert labels dataframe to numpy array 
labels = labelsdf.to_numpy()

# write labels.h5 file 
h5l = h5py.File('label_data.h5', 'w')
h5l.create_dataset('label_data', data=labels)
h5l.close()

NameError: name 'labelsdf' is not defined